# Traitement amélioré des catégories

Ce notebook présenté des encoding différents de ceux implémentées dans [scikit-learn](http://scikit-learn.org/stable/).

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline

On construit un jeu très simple avec deux catégories, une entière, une au format texte.

In [3]:
import pandas
import numpy
df = pandas.DataFrame(dict(cat_int=[10, 20, 10, 39, 10, 10, numpy.nan],
                          cat_text=['catA', 'catB', 'catA', 'catDD', 'catB', numpy.nan, 'catB']))
df

,cat_int,cat_text
0,10.0,catA
1,20.0,catB
2,10.0,catA
3,39.0,catDD
4,10.0,catB
5,10.0,NaN
6,NaN,catB


## Une API un peu différente

Le module [Category Encoders](http://contrib.scikit-learn.org/categorical-encoding/index.html) implémente d'autres options avec une [API](http://contrib.scikit-learn.org/categorical-encoding/onehot.html) un peu différente puisqu'il est possible de spécifier la colonne sur laquelle s'applique l'encoding.

In [4]:
from category_encoders import OneHotEncoder
OneHotEncoder(cols=['cat_text']).fit_transform(df)

,cat_text_0,cat_text_1,cat_text_2,cat_text_3,cat_text_-1,cat_int
0,1,0,0,0,0,10.0
1,0,1,0,0,0,20.0
2,1,0,0,0,0,10.0
3,0,0,1,0,0,39.0
4,0,1,0,0,0,10.0
5,0,0,0,0,1,10.0
6,0,1,0,0,0,NaN


## Autres options

In [5]:
import category_encoders
encoders = []
for k, enc in category_encoders.__dict__.items():
    if 'Encoder' in k:
        encoders.append(enc)
encoders

[category_encoders.backward_difference.BackwardDifferenceEncoder,
 category_encoders.binary.BinaryEncoder,
 category_encoders.hashing.HashingEncoder,
 category_encoders.helmert.HelmertEncoder,
 category_encoders.one_hot.OneHotEncoder,
 category_encoders.ordinal.OrdinalEncoder,
 category_encoders.sum_coding.SumEncoder,
 category_encoders.polynomial.PolynomialEncoder,
 category_encoders.basen.BaseNEncoder,
 category_encoders.leave_one_out.LeaveOneOutEncoder,
 category_encoders.target_encoder.TargetEncoder]

In [6]:
for encoder in encoders:
    if 'Leave' in encoder.__name__ or 'Target' in encoder.__name__:
        continue
    enc = encoder(cols=['cat_text'])
    out = enc.fit_transform(df)
    print('-----', encoder.__name__)
    print(out)
    print('-----')    

----- BackwardDifferenceEncoder
   col_cat_text_0  col_cat_text_1  col_cat_text_2  col_cat_text_3  col_cat_int
0             1.0            0.25            -0.5           -0.25         10.0
1             1.0            0.25             0.5           -0.25         20.0
2             1.0            0.25            -0.5           -0.25         10.0
3             1.0            0.25             0.5            0.75         39.0
4             1.0            0.25             0.5           -0.25         10.0
5             1.0           -0.75            -0.5           -0.25         10.0
6             1.0            0.25             0.5           -0.25          NaN
-----
----- BinaryEncoder
   cat_text_0  cat_text_1  cat_int
0         0.0         0.0     10.0
1         0.0         1.0     20.0
2         0.0         0.0     10.0
3         1.0         0.0     39.0
4         0.0         1.0     10.0
5         NaN         NaN     10.0
6         0.0         1.0      NaN
-----
----- HashingEncoder
   

## Utilisation de la cible

Certains encoding optimise l'encoding en fonction de la cible à prédire lors d'un apprentissage supervisé. Les deux encoders suivant prédisent la cible en fonction de la catégorie ou essayent d'optimiser l'encoding de la catégorie en fonction de la cible à prédire. En particulier, l'encoder [LeaveOneOut](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html) associe à chaque modéalité la moyenne des valeurs observées sur une autre colonne pour chaque ligne associée à cette modalité.

In [7]:
dfy = df.sort_values('cat_text').reset_index(drop=True).copy()
dfy['cat_text_copy'] = dfy['cat_text']
dfy['y'] = dfy.index * dfy.index + 10
dfy['y_copy'] = dfy.y
dfy

,cat_int,cat_text,cat_text_copy,y,y_copy
0,10.0,catA,catA,10,10
1,10.0,catA,catA,11,11
2,20.0,catB,catB,14,14
3,10.0,catB,catB,19,19
4,NaN,catB,catB,26,26
5,39.0,catDD,catDD,35,35
6,10.0,NaN,NaN,46,46


In [8]:
for encoder in encoders:
    enc = encoder(cols=['cat_text'])
    try:
        out = enc.fit_transform(dfy.drop('y', axis=1))
    except Exception as e:
        out = pandas.DataFrame()
    outy = enc.fit_transform(dfy.drop('y', axis=1), dfy.y)
    if not out.equals(outy):
        print('-----', encoder.__name__)
        print(outy)
        print('-----')  

----- LeaveOneOutEncoder
   cat_int cat_text_copy  y_copy   cat_text
0     10.0          catA      10  10.500000
1     10.0          catA      11  10.500000
2     20.0          catB      14  19.666667
3     10.0          catB      19  19.666667
4      NaN          catB      26  19.666667
5     39.0         catDD      35  35.000000
6     10.0           NaN      46  23.000000
-----
----- TargetEncoder
   cat_int cat_text_copy  y_copy   cat_text
0     10.0          catA      10  13.861768
1     10.0          catA      11  13.861768
2     20.0          catB      14  20.064010
3     10.0          catB      19  20.064010
4      NaN          catB      26  20.064010
5     39.0         catDD      35  23.000000
6     10.0           NaN      46  23.000000
-----
